# 3 - Análise Exploratória da Fonte Espacenet

# 3.1: Carregando e Explorando os Dados da Espacenet

In [ ]:
import pandas as pd
from pathlib import Path

print("--- Célula 1: Iniciando a análise exploratória da fonte Espacenet ---")

# Define o caminho para a pasta de dados.
caminho_pasta = Path('../../data/raw/espacenet_input')

try:
    # Encontra todos os arquivos CSV na pasta
    arquivos_csv = list(caminho_pasta.glob('*.csv'))
    
    if not arquivos_csv:
        print(f"ERRO: Nenhum arquivo CSV encontrado na pasta '{caminho_pasta.resolve()}'")
    else:
        print(f"Encontrados {len(arquivos_csv)} arquivos CSV. Carregando e consolidando...")
        
        # CORREÇÃO: Trocando o separador para ponto e vírgula (';')
        lista_dfs = [pd.read_csv(f, sep=';') for f in arquivos_csv]
        
        # Concatena todos os DataFrames em um só
        df_espacenet = pd.concat(lista_dfs, ignore_index=True)

        print(f"\nArquivos consolidados com sucesso! Total de {len(df_espacenet)} registros.")
        print("-" * 50)
        
        # 1. Informações Gerais
        print("\n[1] INFORMAÇÕES GERAIS DO DATAFRAME:")
        df_espacenet.info()
        print("-" * 50)

        # 2. Amostra dos Dados
        print("\n[2] AMOSTRA DOS DADOS (5 PRIMEIRAS LINHAS):")
        display(df_espacenet.head())
        print("-" * 50)
        
        # 3. Verificação de Valores Nulos
        print("\n[3] CONTAGEM DE VALORES NULOS POR COLUNA:")
        valores_nulos = df_espacenet.isnull().sum()
        print(valores_nulos[valores_nulos > 0].sort_values(ascending=False))
        print("-" * 50)

except FileNotFoundError:
    print(f"\nERRO: Pasta não encontrada em '{caminho_pasta.resolve()}'")
except Exception as e:
    print(f"\nOcorreu um erro inesperado ao carregar os arquivos: {e}")
    print("Dica: Se o erro persistir, pode ser um problema de codificação (tente adicionar encoding='latin-1').")

# 3.2: Limpeza Inicial do DataFrame Espacenet

In [ ]:
import unicodedata
import re

print("--- Célula 2: Limpeza estrutural do DataFrame da Espacenet ---")

try:
    df_espacenet_limpo = df_espacenet.copy()

    # --- 1. Remover as colunas 'Unnamed' ---
    colunas_unnamed = [col for col in df_espacenet_limpo.columns if 'unnamed' in col.lower()]
    df_espacenet_limpo = df_espacenet_limpo.drop(columns=colunas_unnamed)
    print(f"[1] Removidas {len(colunas_unnamed)} colunas 'Unnamed'.")

    # --- 2. Padronizar os nomes das colunas restantes ---
    print("\n[2] Padronizando nomes das colunas...")
    print("   Colunas ANTES:", df_espacenet_limpo.columns.tolist())
    
    novas_colunas = []
    for col in df_espacenet_limpo.columns:
        col = str(col)
        col_normalizada = unicodedata.normalize('NFKD', col).encode('ascii', 'ignore').decode('utf-8')
        col_minuscula = col_normalizada.lower()
        col_com_underscore = col_minuscula.replace(' ', '_').replace('-', '_')
        col_final = re.sub(r'[^a-z0-9_]', '', col_com_underscore)
        novas_colunas.append(col_final)
    df_espacenet_limpo.columns = novas_colunas
    
    print("   Colunas DEPOIS:", df_espacenet_limpo.columns.tolist())

    # --- 3. Tratamento básico de nulos para evitar erros futuros ---
    # Converte colunas que serão multivaloradas para string e preenche nulos
    colunas_para_tratar = ['inventors', 'applicants', 'ipc', 'cpc', 'publication_number', 'publication_date']
    for col in colunas_para_tratar:
        if col in df_espacenet_limpo.columns:
            df_espacenet_limpo[col] = df_espacenet_limpo[col].astype(str).fillna('NAO INFORMADO')

    print("\n[3] Limpeza básica de nulos concluída.")
    
    print("\n--- Limpeza inicial finalizada! ---")
    print("DataFrame 'df_espacenet_limpo' foi criado.")
    display(df_espacenet_limpo.head())
    
except NameError:
    print("ERRO: O DataFrame 'df_espacenet' não foi encontrado. Execute a Célula 1 primeiro.")

# 3.3: Criando a Dimensão de Parties (Inventores e Requerentes)

In [ ]:
print("--- Célula 3: Criando a dimensão de Parties e a tabela ponte ---")

try:
    # 1. Selecionar colunas de interesse e garantir que sejam strings
    df_parties_trab = df_espacenet_limpo[['publication_number', 'inventors', 'applicants']].copy()
    
    # 2. Despivôar (unpivot/melt) as colunas de inventors e applicants
    # para ter uma única coluna com o nome e outra com o "papel" (role)
    df_melted = df_parties_trab.melt(
        id_vars=['publication_number'],
        value_vars=['inventors', 'applicants'],
        var_name='role',
        value_name='party_string'
    )
    
    # Remove linhas onde não há informação de party
    df_melted = df_melted[df_melted['party_string'] != 'NAO INFORMADO']

    # 3. Separar os múltiplos nomes (que estão separados por vírgula) e explodir
    df_melted['party_string'] = df_melted['party_string'].str.split(',')
    df_explodido = df_melted.explode('party_string')
    
    # Limpa espaços extras de cada nome
    df_explodido['party_string'] = df_explodido['party_string'].str.strip()
    df_explodido = df_explodido[df_explodido['party_string'] != '']

    # 4. Criar a tabela de dimensão com parties únicas
    # Usamos o nome limpo como a entidade única para a dimensão
    dim_parties = pd.DataFrame(df_explodido['party_string'].unique(), columns=['party_nome'])
    dim_parties.reset_index(inplace=True)
    dim_parties.rename(columns={'index': 'party_id'}, inplace=True)

    print("\n[1] Tabela 'dim_parties' criada:")
    display(dim_parties.head())

    # 5. Criar a tabela ponte
    pon_patente_party = pd.merge(
        df_explodido,
        dim_parties,
        left_on='party_string',
        right_on='party_nome',
        how='left'
    )
    
    # A tabela ponte agora pode ter o papel (inventor/applicant)
    # Primeiro, vamos simplificar o nome do papel
    pon_patente_party['role'] = pon_patente_party['role'].str.replace('s', '') # inventors -> inventor
    
    # Seleciona e agrupa para garantir uma relação única por papel
    pon_patente_party = pon_patente_party[['publication_number', 'party_id', 'role']].drop_duplicates()

    print("\n[2] Tabela 'pon_patente_party' criada:")
    display(pon_patente_party.head())

except NameError:
    print("ERRO: O DataFrame 'df_espacenet_limpo' não foi encontrado. Execute a Célula 2 primeiro.")

# 3.4: Criando a Dimensão de Países

In [ ]:
print("--- Célula 4: Criando a dimensão de Países e a tabela ponte ---")

# Dicionário de códigos de país, extraído do seu script original
COUNTRY_CODES = {
    'AR': 'Argentina', 'AT': 'Áustria', 'AU': 'Austrália', 'BA': 'Bósnia e Herzegovina', 'BE': 'Bélgica', 
    'BG': 'Bulgária', 'BR': 'Brasil', 'CA': 'Canadá', 'CH': 'Suíça', 'CN': 'China', 
    'CS': 'Checoeslováquia (até 1993)', 'CU': 'Cuba', 'CY': 'Chipre', 'CZ': 'República Checa', 
    'DD': 'Alemanha Oriental', 'DE': 'Alemanha', 'DK': 'Dinamarca', 'DZ': 'Argélia', 
    'EA': 'Organização Euroasiática de Patentes', 'EE': 'Estónia', 'EG': 'Egipto', 
    'EP': 'Organização Europeia de Patentes (OPE/EPO)', 'ES': 'Espanha', 'FI': 'Finlândia', 
    'FR': 'França', 'GB': 'Reino Unido', 'GR': 'Grécia', 'HK': 'Hong Kong', 'HU': 'Hungria', 
    'IE': 'Irlanda', 'IL': 'Israel', 'IN': 'India', 'IT': 'Itália', 'JP': 'Japão', 'KE': 'Quénia', 
    'KR': 'Coreia do Sul', 'LU': 'Luxemburgo', 'LV': 'Letónia', 'MC': 'Mónaco', 'MD': 'República da Moldávia', 
    'MN': 'Mongólia', 'MT': 'Malta', 'MW': 'Malawi', 'MX': 'México', 'MY': 'Malásia', 'NC': 'Nova Caledónia', 
    'NL': 'Holanda', 'NO': 'Noruega', 'NZ': 'Nova Zelândia', 
    'OA': 'Organização Africana da Propriedade Intelectual (OAPI)', 'PH': 'Filipinas', 'PL': 'Polónia', 
    'PT': 'Portugal', 'RO': 'Roménia', 'RU': 'Federação Russa', 'SE': 'Suécia', 'SG': 'Singapura', 
    'SI': 'Eslovénia', 'SK': 'Eslováquia', 'SU': 'União Soviética', 'TJ': 'Tadjiquistão', 'TR': 'Turquia', 
    'TT': 'Trindade e Tobago', 'TW': 'Taiwan', 'US': 'Estados Unidos da América', 'VN': 'Vietname', 
    'WO': 'Organização Mundial da Propriedade Intelectual (OMPI/WIPO)', 'YU': 'Jugoslávia', 
    'ZA': 'África do Sul', 'ZM': 'Zâmbia', 'ZW': 'Zimbabwe'
}

try:
    # --- 1. Criar a tabela dim_country ---
    dim_country = pd.DataFrame(COUNTRY_CODES.items(), columns=['country_code', 'country_name'])
    dim_country.reset_index(inplace=True)
    dim_country.rename(columns={'index': 'country_id'}, inplace=True)
    print("\n[1] Tabela 'dim_country' criada:")
    display(dim_country.head())

    # --- 2. Extrair códigos de país de múltiplas fontes ---
    # Fonte 1: Número da Publicação
    df_from_pub = df_espacenet_limpo[['publication_number']].copy()
    df_from_pub['country_code'] = df_from_pub['publication_number'].str[:2]
    df_from_pub['origin'] = 'Publication'
    
    # Fonte 2 e 3: Requerentes (Applicants) e Inventores (Inventors)
    df_from_parties = df_espacenet_limpo[['publication_number', 'inventors', 'applicants']].copy()
    df_melted_parties = df_from_parties.melt(id_vars=['publication_number'], value_vars=['inventors', 'applicants'], var_name='origin')
    df_melted_parties['value'] = df_melted_parties['value'].str.split(',')
    df_exploded_parties = df_melted_parties.explode('value')
    # Extrai o código de 2 letras entre colchetes
    df_exploded_parties['country_code'] = df_exploded_parties['value'].str.extract(r'\[([A-Z]{2})\]')
    df_exploded_parties['origin'] = df_exploded_parties['origin'].str.replace('s', '').str.capitalize()
    
    # --- 3. Consolidar todas as fontes e criar a tabela ponte ---
    # Juntamos os códigos extraídos das publicações e das parties
    df_relations = pd.concat([
        df_from_pub[['publication_number', 'country_code', 'origin']],
        df_exploded_parties[['publication_number', 'country_code', 'origin']]
    ]).dropna(subset=['country_code'])

    # Filtra apenas relações com códigos de país que existem em nosso dicionário
    df_relations = df_relations[df_relations['country_code'].isin(COUNTRY_CODES.keys())]

    # Junta com a dim_country para obter o country_id
    pon_patente_country = pd.merge(
        df_relations,
        dim_country,
        on='country_code',
        how='left'
    )
    # Seleciona as colunas finais e remove duplicatas
    pon_patente_country = pon_patente_country[['publication_number', 'country_id', 'origin']].drop_duplicates()

    print("\n[2] Tabela 'pon_patente_country' criada:")
    display(pon_patente_country.head())

except NameError:
    print("ERRO: O DataFrame 'df_espacenet_limpo' não foi encontrado. Execute a Célula 2 primeiro.")

# 3.5: Criando a Dimensão de Classificação IPC

In [ ]:
print("--- Célula 5: Criando a dimensão de Classificação IPC e a tabela ponte ---")

# --- 1. Função Auxiliar de Limpeza (adaptada do seu script) ---
def clean_ipc_code(ipc_code_str):
    """
    Padroniza um código IPC para um formato consistente, removendo espaços e hífens.
    """
    if pd.isna(ipc_code_str) or not isinstance(ipc_code_str, str):
        return None
    # Converte para maiúsculas e remove espaços/hífens
    return ipc_code_str.upper().strip().replace('-', '').replace(' ', '')

try:
    # --- 2. Preparar e explodir os dados de IPC ---
    df_ipc_trab = df_espacenet_limpo[['publication_number', 'ipc']].copy()
    df_ipc_trab = df_ipc_trab[df_ipc_trab['ipc'] != 'NAO INFORMADO']

    # Divide os códigos IPC (separados por vírgula) em listas
    df_ipc_trab['ipc'] = df_ipc_trab['ipc'].str.split(',')
    df_ipc_explodido = df_ipc_trab.explode('ipc')
    
    # Aplica a função de limpeza para padronizar os códigos
    df_ipc_explodido['ipc_code_normalizado'] = df_ipc_explodido['ipc'].apply(clean_ipc_code)
    df_ipc_explodido.dropna(subset=['ipc_code_normalizado'], inplace=True)
    df_ipc_explodido = df_ipc_explodido[df_ipc_explodido['ipc_code_normalizado'] != '']
    
    # --- 3. Criar a tabela de dimensão IPC ---
    # A dimensão será baseada nos códigos normalizados únicos
    dim_ipc = pd.DataFrame(df_ipc_explodido['ipc_code_normalizado'].unique(), columns=['ipc_code'])
    dim_ipc.reset_index(inplace=True)
    dim_ipc.rename(columns={'index': 'ipc_id'}, inplace=True)

    print("\n[1] Tabela 'dim_ipc' criada:")
    display(dim_ipc.head())

    # --- 4. Criar a tabela ponte Patente-IPC ---
    pon_patente_ipc = pd.merge(
        df_ipc_explodido,
        dim_ipc,
        left_on='ipc_code_normalizado',
        right_on='ipc_code',
        how='left'
    )
    pon_patente_ipc = pon_patente_ipc[['publication_number', 'ipc_id']].drop_duplicates()

    print("\n[2] Tabela 'pon_patente_ipc' criada:")
    display(pon_patente_ipc.head())

except NameError:
    print("ERRO: O DataFrame 'df_espacenet_limpo' não foi encontrado. Execute a Célula 2 primeiro.")

# 3.6:  Carregando seu Levantamento Manual de Espécies

In [ ]:
print("--- Célula 6: Carregando o arquivo de levantamento manual de espécies da Espacenet ---")

# Define o caminho para o seu novo arquivo
caminho_manual = Path('../../data/raw/espacenet_resumo_plantas.csv')

try:
    # Carrega o arquivo. Vamos assumir o separador padrão (vírgula)
    df_espacenet_especies_manual = pd.read_csv(caminho_manual)
    
    print(f"\nArquivo '{caminho_manual.name}' carregado com sucesso!")
    print("-" * 50)
    
    # 1. Informações Gerais
    print("\n[1] INFORMAÇÕES GERAIS DO DATAFRAME MANUAL:")
    df_espacenet_especies_manual.info()
    print("-" * 50)

    # 2. Amostra dos Dados
    print("\n[2] AMOSTRA DOS DADOS MANUAIS (5 PRIMEIRAS LINHAS):")
    display(df_espacenet_especies_manual.head())
    print("-" * 50)

except FileNotFoundError:
    print(f"\nERRO: Arquivo não encontrado em '{caminho_manual.resolve()}'")
    print("Por favor, verifique se o nome do arquivo e o caminho estão corretos.")
except Exception as e:
    print(f"\nOcorreu um erro inesperado ao carregar o arquivo: {e}")

# 3.7: Mapeamento Automático de Espécies (para Comparação)

In [11]:
import re

print("--- Célula 7: Realizando o mapeamento automático de espécies para validação ---")

try:
    # --- NOVO: Carregar o df_termos necessário para o dicionário de busca ---
    caminho_txt_cncflora = Path('../../data/raw/cncflora/termos_plantas.txt')
    df_termos = pd.read_csv(
        caminho_txt_cncflora, 
        sep=',', 
        header=None, 
        names=['nome_cientifico_completo', 'grupo_taxonomico']
    )
    print("[0] DataFrame 'df_termos' da CNCFlora carregado para consulta.")
    # --- Fim da adição ---

    # Padronizar as colunas do arquivo manual primeiro
    df_manual_padronizado = df_espacenet_especies_manual.copy()
    df_manual_padronizado.columns = [
        'publication_number', 
        'abstract', 
        'planta_levantada_manualmente'
    ]
    
    # --- 1. Enriquecer o df_espacenet_limpo com os resumos do arquivo manual ---
    df_resumos = df_manual_padronizado[['publication_number', 'abstract']].copy()
    
    df_espacenet_enriquecido = pd.merge(
        df_espacenet_limpo,
        df_resumos.drop_duplicates(subset=['publication_number']),
        on='publication_number',
        how='left'
    )
    df_espacenet_enriquecido['abstract'] = df_espacenet_enriquecido['abstract'].fillna('NAO INFORMADO')
    print("[1] DataFrame da Espacenet enriquecido com os resumos.")

    # --- 2. Criar o dicionário de busca unificado ---
    plantas_cncflora = df_termos['nome_cientifico_completo'].dropna().unique()
    plantas_manual_espacenet = df_manual_padronizado['planta_levantada_manualmente'].dropna().unique()
    
    lista_de_plantas_mestre = pd.concat([
        pd.Series(plantas_cncflora),
        pd.Series(plantas_manual_espacenet)
    ]).dropna().unique().tolist()
    
    regex_pattern = r'\b(' + '|'.join(re.escape(nome) for nome in lista_de_plantas_mestre) + r')\b'
    print(f"[2] Dicionário de busca com {len(lista_de_plantas_mestre)} nomes de plantas foi criado.")

    # --- 3. Executar a busca automática ---
    df_espacenet_enriquecido['texto_busca'] = (df_espacenet_enriquecido['title'] + ' ' + df_espacenet_enriquecido['abstract'])
    df_espacenet_enriquecido['especies_encontradas_auto'] = df_espacenet_enriquecido['texto_busca'].str.findall(regex_pattern, flags=re.IGNORECASE)
    print("[3] Busca automática por espécies concluída.")

    # --- 4. Exibir tabela de comparação ---
    df_comparacao = pd.merge(
        df_espacenet_enriquecido,
        df_manual_padronizado, 
        on='publication_number',
        how='left'
    )
    
    colunas_comparacao = ['publication_number', 'planta_levantada_manualmente', 'especies_encontradas_auto', 'title']
    print("\n--- Tabela de Comparação (Manual vs. Automático) ---")
    display(df_comparacao[
        (df_comparacao['planta_levantada_manualmente'].notna()) | 
        (df_comparacao['especies_encontradas_auto'].apply(len) > 0)
    ][colunas_comparacao].head(10))

except NameError as e:
    print(f"ERRO: Um dos DataFrames necessários não foi encontrado. Certifique-se de que todas as células anteriores foram executadas. Detalhe: {e}")

--- Célula 7: Realizando o mapeamento automático de espécies para validação ---
[0] DataFrame 'df_termos' da CNCFlora carregado para consulta.
[1] DataFrame da Espacenet enriquecido com os resumos.
[2] Dicionário de busca com 1980 nomes de plantas foi criado.
[3] Busca automática por espécies concluída.

--- Tabela de Comparação (Manual vs. Automático) ---


,publication_number,planta_levantada_manualmente,especies_encontradas_auto,title
0,US2018206423A1,Salvinia sprucei,[],MARINE BIOMASS REACTOR AND METHODS RELATED THE...
2,US2010292280A1,Petrea insignis,[],ANTI-PYRETIC VASODILATORS
3,JP2001122732A,Manilkara excelsa,[],COSMETIC COMPOSITION CONTAINING MOISTURIZING P...
6,JPH10139680A,Euxylophora paraensis,[],ANTIANDROGEN AGENT
7,JP2001055318A,Euxylophora paraensis,[],COMPOSITION FOR ORAL CAVITY
8,US2009223000A1,Euxylophora paraensis,[Euxylophora paraensis],Manufacturing process of vegetable colorant ex...
10,US2020390841A1,"Vismia cauliflora, Vismia cavalcantei, Calyco...",[],COMPOSITIONS AND METHODS FOR TREATING WOUNDS
11,US5177306A,Rhabdodendron macrophyllum,[],ABIENOL-PRODUCING SOMACLONAL VARIANTS OF NICOT...
14,JP2012126715A,Passiflora riparia,[],MEDICINE
20,JP2016183142A,Passiflora riparia,[],LIPOSOME FORMULATION OF POLYPHENOL AND PRODUCT...


# 3.8: Construindo o Modelo Final (Tabela Fato e Ponte de Espécies)

In [16]:
print("--- Célula 8: Finalizando o modelo de dados da Espacenet (com pontes de datas) ---")

try:
    # --- 1. Conectar com a Dimensão Mestre de Espécies (lógica inalterada) ---
    print("\n[1] Conectando patentes com a dimensão mestre de espécies...")
    caminho_dim_especies = Path('../../data/processed/dim_especies_mestre.csv')
    dim_especies_mestre = pd.read_csv(caminho_dim_especies)
    lista_de_plantas = dim_especies_mestre['nome_cientifico'].dropna().unique().tolist()
    regex_pattern = r'\b(' + '|'.join(re.escape(nome) for nome in lista_de_plantas) + r')\b'
    
    df_espacenet_enriquecido['especies_encontradas'] = df_espacenet_enriquecido['title'].str.findall(regex_pattern, flags=re.IGNORECASE)
    
    df_com_especies = df_espacenet_enriquecido[df_espacenet_enriquecido['especies_encontradas'].apply(len) > 0]
    pon_patente_especie_temp = df_com_especies[['publication_number', 'especies_encontradas']].explode('especies_encontradas')
    pon_patente_especie_temp.rename(columns={'especies_encontradas': 'nome_cientifico'}, inplace=True)
    
    pon_patente_especie = pd.merge(
        pon_patente_especie_temp, dim_especies_mestre,
        on='nome_cientifico', how='left'
    )[['publication_number', 'especie_id']].drop_duplicates()
    print("Tabela ponte 'pon_patente_especie' criada.")

    # --- 2. Criar Pontes para os Anos das Datas ---
    print("\n[2] Criando tabelas ponte para os anos das colunas de data...")
    colunas_de_data = {
        'earliest_priority': 'ano_prioridade',
        'publication_date': 'ano_publicacao',
        'earliest_publication': 'ano_primeira_publicacao'
    }
    
    tabelas_ponte_data = {} 

    for col_original, nome_ponte in colunas_de_data.items():
        if col_original in df_espacenet_enriquecido.columns:
            df_data_trab = df_espacenet_enriquecido[['publication_number', col_original]].copy()
            df_data_trab.dropna(subset=[col_original], inplace=True)
            
            # CORREÇÃO: Garante que a coluna é do tipo string antes de usar .str.split()
            df_data_trab[col_original] = df_data_trab[col_original].astype(str)
            
            # Divide a string por vírgula e explode
            df_data_trab[col_original] = df_data_trab[col_original].str.split(',')
            df_data_explodido = df_data_trab.explode(col_original)
            
            # Extrai o ano de cada data individual
            df_data_explodido['ano'] = pd.to_datetime(df_data_explodido[col_original], errors='coerce').dt.year
            df_data_explodido.dropna(subset=['ano'], inplace=True)
            df_data_explodido['ano'] = df_data_explodido['ano'].astype(int)

            # Cria a tabela ponte
            ponte_df = df_data_explodido[['publication_number', 'ano']].drop_duplicates()
            tabelas_ponte_data[f'pon_patente_{nome_ponte}'] = ponte_df
            
            print(f"Tabela ponte 'pon_patente_{nome_ponte}' criada.")
            display(ponte_df.head())

    # --- 3. Criar a Tabela Fato Final ---
    print("\n[3] Criando a tabela fato final 'fato_patentes_espacenet'...")
    
    colunas_para_remover = [
        'inventors', 'applicants', 'ipc', 'cpc', 
        'especies_encontradas', 'texto_busca', 'abstract',
        'earliest_priority', 'publication_date', 'earliest_publication'
    ]
    colunas_existentes_para_remover = [col for col in colunas_para_remover if col in df_espacenet_enriquecido.columns]
    
    fato_patentes_espacenet = df_espacenet_enriquecido.drop(columns=colunas_existentes_para_remover)

    print("Tabela 'fato_patentes_espacenet' finalizada:")
    display(fato_patentes_espacenet.head())

except Exception as e:
    print(f"Ocorreu um erro inesperado. Detalhe: {e}")

--- Célula 8: Finalizando o modelo de dados da Espacenet (com pontes de datas) ---

[1] Conectando patentes com a dimensão mestre de espécies...
Tabela ponte 'pon_patente_especie' criada.

[2] Criando tabelas ponte para os anos das colunas de data...
Tabela ponte 'pon_patente_ano_prioridade' criada.


,publication_number,ano
0,US2018206423A1,2017
1,"US10781458B2,US2020010514A1",2014
2,US2010292280A1,2007
3,JP2001122732A,1999
4,"US2010119627A1,US8329228B2",2006


Tabela ponte 'pon_patente_ano_publicacao' criada.


,publication_number,ano
0,US2018206423A1,2018
2,US2010292280A1,2010
3,JP2001122732A,2001
6,JPH10139680A,1998
7,JP2001055318A,2001


Tabela ponte 'pon_patente_ano_primeira_publicacao' criada.


,publication_number,ano
0,US2018206423A1,2018
1,"US10781458B2,US2020010514A1",2016
2,US2010292280A1,2008
3,JP2001122732A,2001
4,"US2010119627A1,US8329228B2",2008



[3] Criando a tabela fato final 'fato_patentes_espacenet'...
Tabela 'fato_patentes_espacenet' finalizada:


,no,title,publication_number,family_number,especies_encontradas_auto
0,1,MARINE BIOMASS REACTOR AND METHODS RELATED THE...,US2018206423A1,61189536,[]
1,2,INSECTICIDAL PROTEINS AND METHODS FOR THEIR USE,"US10781458B2,US2020010514A1",54396965,[]
2,1,ANTI-PYRETIC VASODILATORS,US2010292280A1,39864457,[]
3,2,COSMETIC COMPOSITION CONTAINING MOISTURIZING P...,JP2001122732A,17922236,[]
4,3,SPECIAL EXTRACT AND USE THEREOF FOR INHIBITING...,"US2010119627A1,US8329228B2",38121795,[]


# 3.9: Salvando Todas as Tabelas do Modelo Espacenet

In [17]:
print("--- Célula 9: Salvando os resultados do modelo dimensional da Espacenet ---")

try:
    # 1. Definir o caminho da pasta de saída
    caminho_saida = Path('../../data/processed/espacenet')
    
    # Cria a pasta de saída, se ela não existir
    caminho_saida.mkdir(parents=True, exist_ok=True)
    print(f"[1] Diretório de saída '{caminho_saida}' está pronto.")

    # 2. Criar um dicionário com todas as tabelas a serem salvas
    tabelas_para_salvar = {
        # Tabela Fato
        "fato_patentes_espacenet": fato_patentes_espacenet,
        
        # Dimensões
        "dim_parties": dim_parties,
        "dim_country": dim_country,
        "dim_ipc": dim_ipc,
        
        # Tabelas Ponte
        "pon_patente_party": pon_patente_party,
        "pon_patente_country": pon_patente_country,
        "pon_patente_ipc": pon_patente_ipc,
        "pon_patente_especie": pon_patente_especie
    }
    
    # Adiciona as tabelas ponte de datas ao dicionário
    # A variável 'tabelas_ponte_data' foi criada na Célula 8
    if 'tabelas_ponte_data' in locals():
        tabelas_para_salvar.update(tabelas_ponte_data)

    # 3. Loop para salvar cada tabela
    print("\n[2] Salvando arquivos CSV...")
    for nome_arquivo, df_tabela in tabelas_para_salvar.items():
        if df_tabela is not None:
            caminho_completo = caminho_saida / f"{nome_arquivo}.csv"
            df_tabela.to_csv(caminho_completo, index=False)
            print(f" -> Arquivo salvo: {nome_arquivo}.csv")
        else:
            print(f" -> Aviso: DataFrame para '{nome_arquivo}' não foi gerado e não foi salvo.")
            
    print("\n--- Processo de salvamento concluído! ---")

except NameError as e:
    print(f"ERRO: Um dos DataFrames necessários não foi encontrado. Certifique-se de que todas as células anteriores foram executadas. Detalhe: {e}")

--- Célula 9: Salvando os resultados do modelo dimensional da Espacenet ---
[1] Diretório de saída '../../data/processed/espacenet' está pronto.

[2] Salvando arquivos CSV...
 -> Arquivo salvo: fato_patentes_espacenet.csv
 -> Arquivo salvo: dim_parties.csv
 -> Arquivo salvo: dim_country.csv
 -> Arquivo salvo: dim_ipc.csv
 -> Arquivo salvo: pon_patente_party.csv
 -> Arquivo salvo: pon_patente_country.csv
 -> Arquivo salvo: pon_patente_ipc.csv
 -> Arquivo salvo: pon_patente_especie.csv
 -> Arquivo salvo: pon_patente_ano_prioridade.csv
 -> Arquivo salvo: pon_patente_ano_publicacao.csv
 -> Arquivo salvo: pon_patente_ano_primeira_publicacao.csv

--- Processo de salvamento concluído! ---
